In [1]:
from chgnet.model.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator


In [3]:
model_path = '../Potentials/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'
chgnet = CHGNet.from_file(model_path)


CHGNet v0.3.0 initialized with 412,525 parameters


In [4]:
calc = CHGNetCalculator(chgnet, use_device='mps')

CHGNet will run on mps


In [9]:
#load the structure 
from pymatgen.core import Structure
from pymatgen.io.vasp import Poscar

structure = Structure.from_file('../Archived_Vasp_Jobs/VCrTi_Vacancies_T3_jobs/structure_0_vac_site_0_end_site_7/POSCAR')
#print(structure)

In [19]:
from chgnet.model import StructOptimizer

opt = StructOptimizer(calc, use_device='mps')
relaxed_structure = opt.relax(structure,fmax=0.01,steps=1000,verbose=True,relax_cell=False)

      Step     Time          Energy          fmax
FIRE:    0 17:41:56     -564.201465        0.694551
FIRE:    1 17:41:56     -564.297535        0.696591
FIRE:    2 17:41:57     -564.478741        0.650442
FIRE:    3 17:41:57     -564.717204        0.572166
FIRE:    4 17:41:58     -564.974473        0.493268
FIRE:    5 17:41:58     -565.200199        0.376240
FIRE:    6 17:41:58     -565.354729        0.293174
FIRE:    7 17:41:59     -565.423462        0.193501
FIRE:    8 17:41:59     -565.409823        0.227050
FIRE:    9 17:41:59     -565.412467        0.222065
FIRE:   10 17:41:59     -565.417454        0.213357
FIRE:   11 17:41:59     -565.424603        0.204611
FIRE:   12 17:41:59     -565.433496        0.191588
FIRE:   13 17:42:00     -565.443529        0.179662
FIRE:   14 17:42:00     -565.454043        0.167351
FIRE:   15 17:42:00     -565.464437        0.149228
FIRE:   16 17:42:00     -565.475192        0.132471
FIRE:   17 17:42:00     -565.485286        0.119612
FIRE:   18 17:

In [20]:
print(relaxed_structure['final_structure'])

Full Formula (Ti6 V38 Cr19)
Reduced Formula: Ti6(V2Cr)19
abc   :  10.426946  10.426946  10.426946
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (63)
  #  SP            a          b          c      magmom
---  ----  ---------  ---------  ---------  ----------
  0  Ti     0.498259   0.002579   0.249278  0.0747313
  1  Ti     0.507084   0.25473    0.005285  0.0746262
  2  Ti     0.510438   0.758483   0.499924  0.0729697
  3  Ti     0.742179  -0.006536   0.003313  0.103499
  4  Ti     0.742153   0.241526   0.750912  0.0705516
  5  Ti     0.738101   0.744133   0.24239   0.10424
  6  V      0.008873  -0.011136   0.006442  0.113557
  7  V      0.0164     0.015065   0.494546  0.0681184
  8  V     -0.00645   -0.003677   0.745018  0.128982
  9  V     -0.009383   0.24191   -0.002414  0.0408828
 10  V      0.019963   0.256875   0.270675  0.0666556
 11  V     -0.002971   0.252169   0.502769  0.094786
 12  V      0.008207   0.259422   0.751769  0.0443041
 13